In [56]:
import random

RPS = ["Rock", "Paper", "Scissors"]

class RpsActor():

    def __init__(self):
        self.process = self.step()

    async def step(self):
        while True:
            yield random.choice(RPS)

    async def act(self):
        return await anext(self.process)


class LlmRpsActor():

    def __init__(self, engine):
        self.process = self.step()
        self.engine = engine

    async def step(self):
        while True:
            prompt = "We are playing Rock Paper Scissors\n"
            yield random.choice(RPS)

    async def act(self):
        return await anext(self.process)

In [65]:

player1 = RpsActor()
player2 = RpsActor()

def choose_winner(a1, a2):
    if a1 == a2:
        return None
    return int((RPS.index(a1) + 1) % 3 == RPS.index(a2)) 

wins = [0, 0]
for i in range(3):
    a1 = await player1.act()
    a2 = await player2.act()
    print(a1, a2)
    winner = choose_winner(a1, a2)
    if winner is not None:
        wins[winner] += 1
    player1.observe(a2)
    player2.observe(a1)
wins

Rock Rock
Scissors Paper
Paper Paper


[1, 0]